<a href="https://colab.research.google.com/github/nrj130613/myproject/blob/main/NER_pos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Download Data

In [ ]:
!pip install sklearn-crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.4 MB/s eta 0:00:00


In [ ]:
!gdown 1LHZe1etwxaKnP6TWTbDscasBe5pGykjJ

Downloading...
From: https://drive.google.com/uc?id=1LHZe1etwxaKnP6TWTbDscasBe5pGykjJ
To: /content/train_auto_tok.tsv
100% 38.9M/38.9M [00:00<00:00, 61.0MB/s]


In [ ]:
!head train_auto_tok.tsv

ธรรมนูญ	B_PER
แชมป์	O
สิงห์	O
คลาสสิก	O
กวาด	O
รางวัล	O
แสน	O
สี่	O
หมื่น	O
บาท	O


In [ ]:
!gdown 1LtdB8q2xVhK7vivJxTU6yYnl3KTG-QGS

Downloading...
From: https://drive.google.com/uc?id=1LtdB8q2xVhK7vivJxTU6yYnl3KTG-QGS
To: /content/dev_auto_tok.tsv
100% 3.56M/3.56M [00:00<00:00, 55.0MB/s]


In [ ]:
def load_data(file_name):
    with open(file_name, 'r') as f:
        lines = f.read().splitlines()

    X, Y = [], []
    sentence, labels = [], []
    for line in lines:
        if not line:
            if sentence:
                X.append(sentence)
                Y.append(labels)
                sentence, labels = [], []
        else:
            word, tag = line.split('\t')
            sentence.append(word)
            labels.append(tag)
    if sentence:
        X.append(sentence)
        Y.append(labels)

    return (X, Y)


In [ ]:
Xtrain, Ytrain = load_data('train_auto_tok.tsv')
Xtest, Ytest = load_data('dev_auto_tok.tsv')

In [ ]:
Xtrain

In [ ]:
import sklearn_crfsuite
import sklearn_crfsuite.metrics

# 2. Feature Engineering

In [ ]:
!pip install pythainlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 43.7 MB/s eta 0:00:00


# POS Features


In [ ]:
import pythainlp
from pythainlp.tag import pos_tag

def pos_tagging(sentences):
  sents_pos = []
  for i in range(len(sentences)):
    #for sent in sentences[i]:
    each_sent_pos = pos_tag(sentences[i])

    sents_pos.append(each_sent_pos)
  return sents_pos

In [ ]:
test = pos_tagging(Xtrain[0:5])
test

In [ ]:
def pos_featurize(sent):
  features = {}
  for i in range(len(sent)):
    word, pos = sent[i]
    features["{}_{}_pos".format(word, i)] = pos

  return features

feature_function_list = [pos_featurize]

# 3. Train and evaluate models

In [ ]:
train_set_pos = pos_tagging(Xtrain[0:10])
test_set_pos = pos_tagging(Xtest[0:10])

In [ ]:
import sklearn_crfsuite

In [ ]:
from sklearn_crfsuite import metrics

def train_and_evaluate(Xtrain, Ytrain, Xtest, Ytest, feature_function_list):
    X_train_feats = [pos_featurize(sent) for sent in Xtrain]
    X_test_feats = [pos_featurize(sent) for sent in Xtest]
    crf = sklearn_crfsuite.CRF()
    crf.fit(X_train_feats, Ytrain)
    y_pred = crf.predict(X_test_feats)
    f1_score = metrics.flat_f1_score(Ytest, y_pred, average='weighted', labels=crf.classes_, zero_division=1)
    return f1_score

In [ ]:
train_and_evaluate(train_set_pos, Ytrain[0:10], test_set_pos, Ytest[0:10], [pos_featurize])

0.8692499076448919